# Author : Riasad Alvi
## 15 Jan 2025

In [59]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pima-indians-diabetes-database/diabetes.csv


In [41]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

## Input and Output Col Extract : 

In [60]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [43]:
X

array([[  6.   , 148.   ,  72.   , ...,  33.6  ,   0.627,  50.   ],
       [  1.   ,  85.   ,  66.   , ...,  26.6  ,   0.351,  31.   ],
       [  8.   , 183.   ,  64.   , ...,  23.3  ,   0.672,  32.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,  26.2  ,   0.245,  30.   ],
       [  1.   , 126.   ,  60.   , ...,  30.1  ,   0.349,  47.   ],
       [  1.   ,  93.   ,  70.   , ...,  30.4  ,   0.315,  23.   ]])

## As the data is not scaled , we need to scale the data  : 
### why ? -> NN cant train properly if the data is not scaled . 

### Importing StandardScaler Class : 

In [44]:
from sklearn.preprocessing import StandardScaler 
scaler = StandardScaler()

### Scaling : 

In [45]:
X = scaler.fit_transform(X)

In [46]:
X


array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

### Shape of the input : 

In [48]:
X.shape

(768, 8)

## Doing Train Test Split : 

In [49]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

## Building Model : 

In [51]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [57]:
model = Sequential()

model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [58]:
model.fit(X_train,y_train,batch_size=32,epochs=50,validation_data=(X_test,y_test))

Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.6055 - loss: 0.6603 - val_accuracy: 0.6688 - val_loss: 0.6251
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6641 - loss: 0.6194 - val_accuracy: 0.7013 - val_loss: 0.5895
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7333 - loss: 0.5719 - val_accuracy: 0.7468 - val_loss: 0.5602
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7339 - loss: 0.5465 - val_accuracy: 0.7662 - val_loss: 0.5397
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7570 - loss: 0.5196 - val_accuracy: 0.7532 - val_loss: 0.5224
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7495 - loss: 0.5242 - val_accuracy: 0.7662 - val_loss: 0.5080
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7547 - loss: 0.5106 - val_accuracy: 0.7727 - val_loss: 0.4978
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7690 - loss: 0.4926 - val_accuracy: 0.7727 - val_loss

# Hyper Parameter tunig by Keras Tuner : 
1. How to select appropiate optimizer
2. How to select Number of nodes/neurons in a layer
3. How to select number of layers
4. Implement all in one Model 

In [66]:
# installing kerastuner : 
!pip install -U keras-tuner

In [67]:
import kerastuner as kt

## 1. How to select appropiate optimizer : 

In [74]:
# Building Model Function : 

def build_model(hp): # hp -> hyper parameter

    model = Sequential()
    model.add(Dense(32,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))

    optimizer = hp.Choice('optimizer',values=['adam','rmsprop','sgd']) # optimizers to test 

    model.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=['accuracy']) 


    return model
    

In [75]:
# making tuner obj : 

tuner = kt.RandomSearch(build_model,                # name of the model function 
                       objective = 'val_accuracy',  # our objective is higher val_accuracy
                       max_trials=5)                # how many trials 

Reloading Tuner from ./untitled_project/tuner0.json


In [76]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.7532467246055603

Best val_accuracy So Far: 0.7532467246055603
Total elapsed time: 00h 00m 33s


In [79]:
# best optimizer : 

tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [82]:
# building the model with the best optimizer : 

model = tuner.get_best_models(num_models=1)[0]

In [83]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [86]:
# trainig the model : 

model.fit(X_train,y_train,batch_size=32,epochs=20,initial_epoch=6,validation_data=(X_test,y_test)) # as model already trained for 5 epochs so set initail_epoch=6 (optional)

Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7960 - loss: 0.4341 - val_accuracy: 0.7922 - val_loss: 0.4631
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7690 - loss: 0.4589 - val_accuracy: 0.7922 - val_loss: 0.4622
Epoch 9/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7846 - loss: 0.4482 - val_accuracy: 0.7987 - val_loss: 0.4629
Epoch 10/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7862 - loss: 0.4361 - val_accuracy: 0.7922 - val_loss: 0.4622
Epoch 11/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7916 - loss: 0.4244 - val_accuracy: 0.7857 - val_loss: 0.4623
Epoch 12/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7944 - loss: 0.4155 - val_accuracy: 0.7857 - val_loss: 0.4626
Epoch 13/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7842 - loss: 0.4506 - val_accuracy: 0.7922 - val_loss: 0.4623
Epoch 14/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7990 - loss: 0.4330 - val_accuracy: 0.7922 - val_

# 2. How to Select Appropiate Number of Nodes in a layer : 

In [88]:
# building Model Function: 

def build_model(hp):

    model=Sequential()

    units=hp.Int('units',8,128,step=8)  # hp.Int(lower limit, Upper limit , step size) -> first it will take 8 neurons -> then -> 16 neurons -> then -> 32 neurons

    model.add(Dense(units=units,activation='relu',input_dim=8))

    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

    return model

In [103]:
# making tuner obj : 

tuner = kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=5,
                       directory = 'mydir',      # it will create a folder named mydir containing a sub-folder named new-proj that will contain a json file which will contain the best num of nodes, which will be needed for tuner.search()
                       project_name='new-proj1'
                       )

In [104]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7727272510528564

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 10s


In [106]:
# best number of node  : 

tuner.get_best_hyperparameters()[0].values

{'units': 96}

In [107]:
# building the model with the best number of nodes  :

model = tuner.get_best_models(num_models=1)[0]    # num of nodes = 96

In [108]:
# traing model : 

model.fit(X_train,y_train,batch_size=32,epochs=20,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7541 - loss: 0.5184 - val_accuracy: 0.7857 - val_loss: 0.5021
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7800 - loss: 0.4826 - val_accuracy: 0.7662 - val_loss: 0.4902
Epoch 9/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7605 - loss: 0.4889 - val_accuracy: 0.7597 - val_loss: 0.4849
Epoch 10/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7857 - loss: 0.4534 - val_accuracy: 0.7792 - val_loss: 0.4812
Epoch 11/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7685 - loss: 0.4578 - val_accuracy: 0.7727 - val_loss: 0.4794
Epoch 12/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7776 - loss: 0.4624 - val_accuracy: 0.7792 - val_loss: 0.4793
Epoch 13/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7874 - loss: 0.4505 - val_accuracy: 0.7727 - val_loss: 0.4794
Epoch 14/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7313 - loss: 0.4802 - val_accuracy: 0.7792 - val

# 3. How To Select Appropiate Number of Layers : 


In [110]:
# building Model function : 
def build_model(hp):

    model=Sequential()

    model.add(Dense(96,activation='relu',input_dim=8)) # first hidden layer (receiving inputs)

    for i in range(hp.Int('numOfLayers',min_value=1,max_value=10)):

        model.add(Dense(96,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    return model
        

In [111]:
# making tuner obj : 

tuner=kt.RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=3,
                     directory='mydir',
                     project_name='my-proj2')

In [112]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.8116883039474487

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 00m 10s


In [113]:
# best number of layers  : 

tuner.get_best_hyperparameters()[0].values

{'numOfLayers': 6}

In [115]:
# building the model with the best number of layers :

model = tuner.get_best_models(num_models=1)[0]    # num of layers = 6

In [117]:
# trainin the model : 
model.fit(X_train,y_train,epochs=20,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8733 - loss: 0.2859 - val_accuracy: 0.8247 - val_loss: 0.5815
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8994 - loss: 0.2699 - val_accuracy: 0.7792 - val_loss: 0.6631
Epoch 9/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9009 - loss: 0.2855 - val_accuracy: 0.7338 - val_loss: 0.7923
Epoch 10/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9104 - loss: 0.2392 - val_accuracy: 0.7922 - val_loss: 0.6739
Epoch 11/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9246 - loss: 0.1864 - val_accuracy: 0.7208 - val_loss: 0.8607
Epoch 12/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9315 - loss: 0.2095 - val_accuracy: 0.7208 - val_loss: 0.9527
Epoch 13/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9430 - loss: 0.1639 - val_accuracy: 0.7597 - val_loss: 0.8776
Epoch 14/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9313 - loss: 0.2108 - val_accuracy: 0.7403 - val_

# 4. Implementing all hyper parameter tuning in one model : 

In [139]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

def build_model(hp, input_shape):  # Allow dynamic input shape
    model = Sequential()
    
    for i in range(hp.Int('numOfLayers', min_value=1, max_value=10)):  # Tune number of layers
        if i == 0:  # First layer
            model.add(
                Dense(
                    hp.Int('units' + str(i), min_value=8, max_value=256, step=8),  # Larger range for neurons
                    activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid']),
                    kernel_initializer=hp.Choice('initializer', values=['he_uniform', 'glorot_uniform', 'lecun_normal']),
                    input_shape=input_shape
                )
            )
        else:  # Subsequent layers
            model.add(
                Dense(
                    hp.Int('units' + str(i), min_value=8, max_value=256, step=8),
                    activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid']),
                    kernel_initializer=hp.Choice('initializer', values=['he_uniform', 'glorot_uniform', 'lecun_normal'])
                )
            )
        
        # Add Batch Normalization
        if hp.Boolean('batch_norm' + str(i)):  # Decide whether to use Batch Normalization
            model.add(BatchNormalization())
        
        # Add Dropout Layer
        model.add(Dropout(hp.Float('dropout_rate' + str(i), min_value=0.0, max_value=0.5, step=0.1)))

    # Output layer
    model.add(Dense(1, activation='sigmoid'))

    # Tune optimizer and learning rate
    optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'nadam', 'adadelta'])
    learning_rate = hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='log')

    if optimizer == 'adam':
        optimizer_instance = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        optimizer_instance = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        optimizer_instance = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    elif optimizer == 'nadam':
        optimizer_instance = tf.keras.optimizers.Nadam(learning_rate=learning_rate)
    elif optimizer == 'adadelta':
        optimizer_instance = tf.keras.optimizers.Adadelta(learning_rate=learning_rate)

    # Compile the model
    model.compile(
        optimizer=optimizer_instance,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model


In [140]:
tuner = kt.RandomSearch(
    hypermodel=lambda hp: build_model(hp, input_shape=(8,)),  # Pass the input_shape argument
    objective='val_accuracy',  # Optimize for validation accuracy
    max_trials=4,  # Number of trials (combinations of hyperparameters)
    directory='mydir',  # Directory to save logs/results
    project_name='my-proj4'  # Name of the project
)


In [141]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 4 Complete [00h 00m 08s]
val_accuracy: 0.7597402334213257

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 19s


In [142]:
# best appropiate hyperparameters  : 

tuner.get_best_hyperparameters()[0].values

{'numOfLayers': 8,
 'units0': 120,
 'activation0': 'sigmoid',
 'initializer': 'lecun_normal',
 'batch_norm0': True,
 'dropout_rate0': 0.1,
 'optimizer': 'rmsprop',
 'learning_rate': 0.002738558591516091,
 'units1': 8,
 'activation1': 'relu',
 'batch_norm1': False,
 'dropout_rate1': 0.0,
 'units2': 8,
 'activation2': 'relu',
 'batch_norm2': False,
 'dropout_rate2': 0.0,
 'units3': 8,
 'activation3': 'relu',
 'batch_norm3': False,
 'dropout_rate3': 0.0,
 'units4': 8,
 'activation4': 'relu',
 'batch_norm4': False,
 'dropout_rate4': 0.0,
 'units5': 8,
 'activation5': 'relu',
 'batch_norm5': False,
 'dropout_rate5': 0.0,
 'units6': 8,
 'activation6': 'relu',
 'batch_norm6': False,
 'dropout_rate6': 0.0,
 'units7': 8,
 'activation7': 'relu',
 'batch_norm7': False,
 'dropout_rate7': 0.0}

In [145]:
# building the model with the best number of hyper parameters  :

model = tuner.get_best_models(num_models=1)[0]  

In [146]:
model.fit(X_train,y_train,epochs=30,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.7752 - loss: 0.4540 - val_accuracy: 0.7922 - val_loss: 0.5425
Epoch 8/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7448 - loss: 0.4924 - val_accuracy: 0.8117 - val_loss: 0.5354
Epoch 9/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7974 - loss: 0.4550 - val_accuracy: 0.8052 - val_loss: 0.5193
Epoch 10/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7789 - loss: 0.4584 - val_accuracy: 0.8117 - val_loss: 0.5042
Epoch 11/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8066 - loss: 0.4485 - val_accuracy: 0.8052 - val_loss: 0.5028
Epoch 12/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7870 - loss: 0.4665 - val_accuracy: 0.7792 - val_loss: 0.5142
Epoch 13/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7661 - loss: 0.4655 - val_accuracy: 0.7792 - val_loss: 0.4941
Epoch 14/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7661 - loss: 0.4761 - val_accuracy: 0.8117 - val